CNN modelleri 4 boyutlu bir veri yapısı bekler: (Örnek Sayısı, Yükseklik, Genişlik, Kanal Sayısı).
- 28, 28: Resmin boyutları.
- 1: Kanal sayısı (Grayscale yani siyah-beyaz olduğu için 1. Eğer renkli/RGB olsaydı 3 olacaktı).

In [4]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
#Reshape: Görüntüyü 28,28,1 boyutuna getiriyorum
X_train_cnn = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test_cnn = X_test.reshape(X_test.shape[0], 28, 28, 1)

#Normalize: Görüntüyü 0-1 aralığına getiriyorum
X_train_cnn = X_train_cnn.astype('float32') / 255.0
X_test_cnn = X_test_cnn.astype('float32') / 255.0


In [ ]:
from tensorflow.keras import layers,models

model = models.Sequential([
    layers.Conv2D(21, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(42, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])